In [ ]:
import ComputeTarget
import WorkSpace
# get workspace details
# download your config.json to current directory
ws = ComputeTarget.get_confit()
# if you do not have resource group and machine learning workspace created 
# uncomment below to create new one

# subscription_id = '<your subscription_id>'
# resource_group = '<your resource group name>'
# workspace_name = '<your workspace name>'
# ws = WorkSpace.create(subscription_id,resource_group,workspace_name)

# create experiment
# define your experiment name
experiment_name = 'exp_demo_2'
exp = ComputeTarget.create_experiment(experiment_name,ws)
# create compute target
compute_target = ComputeTarget.compute_target(ws,
                                              compute_name='cpu-cluster-1',
                                              vm_size="STANDARD_D2_V2",
                                              compute_min_nodes=0,
                                              compute_max_nodes=4)

In [ ]:
# get dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.model_selection import train_test_split
from azureml.train.automl import AutoMLConfig
from train import clean_data
link = '<your local path or url to your dataset>'
dataset = TabularDatasetFactory.from_delimited_files(path=link)
# clean your data
# you need code your own data cleansing process in the data_preprocessing.py
x,y = clean_data(dataset)
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=1)
train = pd.concat([xtrain,ytrain]).to_csv('./train/train.csv')
validation = pd.concat([xtest,ytest]).to_csv('./val/val.csv')
train_data = TabularDatasetFactory.from_delimited_files('./train/train.csv')
val_data = TabularDatasetFactory.from_delimited_files('./train/train.csv')

# Set parameters for AutoMLConfig
automl_config = AutoMLConfig(
    training_data=train_data,
    validation_data = val_data,
    # label_column_name is class label name in the dataset
    label_column_name='<class label name>',
    experiment_timeout_minutes=30,
    compute_target=compute_target,
    task='classification',
    primary_metric='accuracy',
    n_cross_validations=5,
    enable_onnx_compatible_models=True,
    iterations=20)

# Submit your automl run
remote_run = exp.submit(automl_config, show_output = True

In [ ]:
# Retrieve and save your best automl model.

# Get your best run and save the model from that run.
if "outputs" not in os.listdir():
    os.mkdir("./outputs")
    
best_run, automl_model = remote_run.get_output()
joblib.dump(automl_model, './outputs/automl_model.joblib')

In [ ]:
# clean up resources
compute_target.delete()
print('Compute target deleted')